In [1]:
import xml.etree.ElementTree as ET
import pandas as pd

tree = ET.parse('calendar_9148_1704824475_2024-01-09.xml')
root = tree.getroot()

data = []

root_events = root.find('events')
events_all = root_events.findall('event')

for event in events_all:
    event_data = {}
    for tag in ['title', 'start', 'end', 'URL', 'categories']:
        event_tag = event.find(tag)
        if event_tag is not None:
            if tag == 'categories':
                categories = [cats.text for cats in event_tag.findall('category')]
                event_data[tag] = ', '.join(categories)
            else:
                event_data[tag] = event_tag.text
        else:
            event_data[tag] = None
    data.append(event_data)

df = pd.DataFrame(data)
df.head()

,title,start,end,URL,categories
0,3MT Slide Design and Presentation Skills Overv...,2024-01-22T16:00:00-05:00,2024-01-22T17:00:00-05:00,https://cmu.libcal.com/event/11620595,Writing and Publishing
1,Submitting Your Thesis or Dissertation to CMU ...,2024-01-24T12:00:00-05:00,2024-01-24T13:00:00-05:00,https://cmu.libcal.com/event/11622573,"Open Science, Writing and Publishing"
2,Open Science Framework (OSF) for Beginners,2024-01-26T13:00:00-05:00,2024-01-26T14:00:00-05:00,https://cmu.libcal.com/event/11721905,Open Science
3,Data Carpentry: Introduction to Python,2024-01-29T09:00:00-05:00,2024-01-29T12:00:00-05:00,https://cmu.libcal.com/event/11732479,"Open Science, Working With Data"
4,Introduction to LaTeX and Overleaf,2024-01-29T12:00:00-05:00,2024-01-29T13:00:00-05:00,https://cmu.libcal.com/event/11622522,Writing and Publishing


In [4]:
df.dropna(inplace=True)

In [5]:
filtered_rows = df[df['categories'].apply(lambda cats: all(cat in cats.split(', ') for cat in ['Open Science']))]
filtered_rows.head()

,title,start,end,URL,categories
1,Submitting Your Thesis or Dissertation to CMU ...,2024-01-24T12:00:00-05:00,2024-01-24T13:00:00-05:00,https://cmu.libcal.com/event/11622573,"Open Science, Writing and Publishing"
2,Open Science Framework (OSF) for Beginners,2024-01-26T13:00:00-05:00,2024-01-26T14:00:00-05:00,https://cmu.libcal.com/event/11721905,Open Science
3,Data Carpentry: Introduction to Python,2024-01-29T09:00:00-05:00,2024-01-29T12:00:00-05:00,https://cmu.libcal.com/event/11732479,"Open Science, Working With Data"
5,Data Carpentry: Introduction to Python,2024-01-30T09:00:00-05:00,2024-01-30T12:00:00-05:00,https://cmu.libcal.com/event/11732480,"Open Science, Working With Data"
7,Introduction to R Part 1: Getting started with...,2024-01-30T13:00:00-05:00,2024-01-30T14:30:00-05:00,https://cmu.libcal.com/event/11677891,"Open Science, Working With Data"


In [6]:
workshops = filtered_rows.copy()

In [8]:
workshops['date'] = pd.to_datetime(workshops['start'], format='%Y-%m-%d', utc=True).dt.date
workshops['formatted_date'] = pd.to_datetime(workshops['date']).dt.strftime('%B %d, %Y')
workshops['start'] = pd.to_datetime(workshops['start'], format='%Y-%m-%d %H:%M:%S', utc=True).dt.tz_convert('US/Eastern').dt.strftime('%I:%M %p')
workshops['end'] = pd.to_datetime(workshops['end'], format='%Y-%m-%d %H:%M:%S', utc=True).dt.tz_convert('US/Eastern').dt.strftime('%I:%M %p')
workshops.head()

,title,start,end,URL,categories,date,formatted_date
1,Submitting Your Thesis or Dissertation to CMU ...,12:00 PM,01:00 PM,https://cmu.libcal.com/event/11622573,"Open Science, Writing and Publishing",2024-01-24,"January 24, 2024"
2,Open Science Framework (OSF) for Beginners,01:00 PM,02:00 PM,https://cmu.libcal.com/event/11721905,Open Science,2024-01-26,"January 26, 2024"
3,Data Carpentry: Introduction to Python,09:00 AM,12:00 PM,https://cmu.libcal.com/event/11732479,"Open Science, Working With Data",2024-01-29,"January 29, 2024"
5,Data Carpentry: Introduction to Python,09:00 AM,12:00 PM,https://cmu.libcal.com/event/11732480,"Open Science, Working With Data",2024-01-30,"January 30, 2024"
7,Introduction to R Part 1: Getting started with...,01:00 PM,02:30 PM,https://cmu.libcal.com/event/11677891,"Open Science, Working With Data",2024-01-30,"January 30, 2024"


In [9]:
workshops.to_csv('workshops.csv', index=False)

In [10]:
paths = pd.read_csv('content_paths.csv').drop('title',axis=1)
paths

,path,URL
0,content/osf.md,https://cmu.libcal.com/event/11721905
1,https://datacarpentry.org/python-ecology-lesson/,https://cmu.libcal.com/event/11732479
2,https://datacarpentry.org/python-ecology-lesson/,https://cmu.libcal.com/event/11732480
3,IntroR_Materials/introR_pt1/00_introR_part1.md,https://cmu.libcal.com/event/11677891
4,https://datacarpentry.org/python-ecology-lesson/,https://cmu.libcal.com/event/11732481
5,Python_Series_Materials/part_1/part_1.md,https://cmu.libcal.com/event/11690015
6,cli/cli_main.md,https://cmu.libcal.com/event/11722733
7,IntroR_Materials/introR_pt2/00_introR_part2.md,https://cmu.libcal.com/event/11677895
8,Python_Series_Materials/part_3/part_3.md,https://cmu.libcal.com/event/11720589
9,Git_Materials/index.md,https://cmu.libcal.com/event/11720611


In [11]:
workshops_paths = pd.merge(left=workshops, right=paths, how='left', left_on='URL', right_on='URL')
workshops_paths.fillna(0, inplace=True)
workshops_paths

,title,start,end,URL,categories,date,formatted_date,path
0,Submitting Your Thesis or Dissertation to CMU ...,12:00 PM,01:00 PM,https://cmu.libcal.com/event/11622573,"Open Science, Writing and Publishing",2024-01-24,"January 24, 2024",0
1,Open Science Framework (OSF) for Beginners,01:00 PM,02:00 PM,https://cmu.libcal.com/event/11721905,Open Science,2024-01-26,"January 26, 2024",content/osf.md
2,Data Carpentry: Introduction to Python,09:00 AM,12:00 PM,https://cmu.libcal.com/event/11732479,"Open Science, Working With Data",2024-01-29,"January 29, 2024",https://datacarpentry.org/python-ecology-lesson/
3,Data Carpentry: Introduction to Python,09:00 AM,12:00 PM,https://cmu.libcal.com/event/11732480,"Open Science, Working With Data",2024-01-30,"January 30, 2024",https://datacarpentry.org/python-ecology-lesson/
4,Introduction to R Part 1: Getting started with...,01:00 PM,02:30 PM,https://cmu.libcal.com/event/11677891,"Open Science, Working With Data",2024-01-30,"January 30, 2024",IntroR_Materials/introR_pt1/00_introR_part1.md
5,Data Carpentry: Introduction to Python,09:00 AM,12:00 PM,https://cmu.libcal.com/event/11732481,"Open Science, Working With Data",2024-01-31,"January 31, 2024",https://datacarpentry.org/python-ecology-lesson/
6,Introduction to Python for Data Science: Intro...,01:00 PM,03:00 PM,https://cmu.libcal.com/event/11690015,"Open Science, Open Science, Working With Data",2024-01-31,"January 31, 2024",Python_Series_Materials/part_1/part_1.md
7,Building your Programming Tool Box: Command Li...,01:00 PM,03:00 PM,https://cmu.libcal.com/event/11722733,"Open Science, Working With Data",2024-02-05,"February 05, 2024",cli/cli_main.md
8,Introduction to R Part 2: Working with data in R,01:00 PM,02:30 PM,https://cmu.libcal.com/event/11677895,"Open Science, Working With Data",2024-02-06,"February 06, 2024",IntroR_Materials/introR_pt2/00_introR_part2.md
9,Introduction to Python for Data Science: Plott...,01:00 PM,03:00 PM,https://cmu.libcal.com/event/11720589,"Open Science, Open Science, Working With Data",2024-02-07,"February 07, 2024",Python_Series_Materials/part_3/part_3.md


In [12]:
table_head = """| Day | Time | Title | Content  
| --- | --- | --- | ---  """

In [13]:
table_body = """
"""

for i, row in workshops_paths.iterrows():
    table_entry = f"| {row['formatted_date']} | {row['start']} - {row['end']} | {row['title']} | [Register Here]({row['URL']}) "
    if row['path'] != 0:
        table_entry += f"[Content]({row['path']}) \n"
    else:
        table_entry += "\n"
    
    table_body += table_entry

In [14]:
print(table_body)


| January 24, 2024 | 12:00 PM - 01:00 PM | Submitting Your Thesis or Dissertation to CMU Libraries | [Register Here](https://cmu.libcal.com/event/11622573) 
| January 26, 2024 | 01:00 PM - 02:00 PM | Open Science Framework (OSF) for Beginners | [Register Here](https://cmu.libcal.com/event/11721905) [Content](content/osf.md) 
| January 29, 2024 | 09:00 AM - 12:00 PM | Data Carpentry: Introduction to Python | [Register Here](https://cmu.libcal.com/event/11732479) [Content](https://datacarpentry.org/python-ecology-lesson/) 
| January 30, 2024 | 09:00 AM - 12:00 PM | Data Carpentry: Introduction to Python | [Register Here](https://cmu.libcal.com/event/11732480) [Content](https://datacarpentry.org/python-ecology-lesson/) 
| January 30, 2024 | 01:00 PM - 02:30 PM | Introduction to R Part 1: Getting started with R and RStudio | [Register Here](https://cmu.libcal.com/event/11677891) [Content](IntroR_Materials/introR_pt1/00_introR_part1.md) 
| January 31, 2024 | 09:00 AM - 12:00 PM | Data Carp

In [15]:
index_md = f"""---
layout: default
title: Home
nav_order: 1
---
# Open Data Science Workshops
Hosted by the Open Science program within the
[Carnegie Mellon University (CMU) Libraries](https://www.library.cmu.edu/)

This website highlights workshops from CMU libraries that help
build computational skills for Open Research and Data Science.

## Workshop Calendar
{table_body}
## Data Office Hours

Schedule a [Data Consultation](https://library.cmu.edu/service/data-office-hours) with our library associates for support with data and research!

![Data Office Hours Flyer](content/img/data-office-hours.png)

## Coordinators

Melanie Gainey (she/her/hers)  
Open Science Program Director/Librarian  
Office: Library 431, Mellon library  
[mgainey@andrew.cmu.edu](mailto:mgainey@andrew.cmu.edu) | [Schedule a Consultation](https://cmu.libcal.com/appointment/42420)

Chasz Griego (he/him/his)  
Science and Engineering Librarian  
Office: 4410, Sorrells Library  
[cgriego@andrew.cmu.edu](mailto:cgriego@andrew.cmu.edu) | [Schedule a Consultation](https://cmu.libcal.com/appointments/cgriego)

Lencia Beltran (she/her/hers)  
Open Science Project Coordinator  
Office: 4416, Sorrells Library  
[lbeltran@andrew.cmu.edu](mailto:lbeltran@andrew.cmu.edu) | [Schedule a Consultation](https://cmu.libcal.com/appointments/lencia)

Emma Slayton (she/her/hers)  
Data Curation, Visualization, and GIS Specialist  
Office: 4408, Sorrells Library  
[eslayton@andrew.cmu.edu](mailto:eslayton@andrew.cmu.edu) | [Schedule a Consultation](https://cmu.libcal.com/appointment/41060)

Sarah Young (she/her/hers)  
Principal Librarian  
Office: 109G, Hunt Library  
[sarahy@andrew.cmu.edu](mailto:sarahy@andrew.cmu.edu) | [Schedule a Consultation](https://cmu.libcal.com/appointments/sarahy)

Emily Bongiovanni (she/her/hers)  
Psychology Librarian Liaison  
Office: 109D, Hunt Library  
[ebongiov@andrew.cmu.edu](mailto:ebongiov@andrew.cmu.edu) | [Schedule a Consultation](https://cmu.libcal.com/appointments/emily)  

Taiwo Lasisi (she/her/hers)  
CLIR Postdoctoral Fellow in Community Data Literacy  
Office: 4418, Sorrells Library  
[tlasisi@andrew.cmu.edu](mailto:tlasisi@andrew.cmu.edu) | [Schedule a Consultation](https://cmu.libcal.com/appointments/TaiwoLasisi)  
"""

In [16]:
print(index_md)

---
layout: default
title: Home
nav_order: 1
---
# Open Data Science Workshops
Hosted by the Open Science program within the
[Carnegie Mellon University (CMU) Libraries](https://www.library.cmu.edu/)

This website highlights workshops from CMU libraries that help
build computational skills for Open Research and Data Science.

## Workshop Calendar

| January 24, 2024 | 12:00 PM - 01:00 PM | Submitting Your Thesis or Dissertation to CMU Libraries | [Register Here](https://cmu.libcal.com/event/11622573) 
| January 26, 2024 | 01:00 PM - 02:00 PM | Open Science Framework (OSF) for Beginners | [Register Here](https://cmu.libcal.com/event/11721905) [Content](content/osf.md) 
| January 29, 2024 | 09:00 AM - 12:00 PM | Data Carpentry: Introduction to Python | [Register Here](https://cmu.libcal.com/event/11732479) [Content](https://datacarpentry.org/python-ecology-lesson/) 
| January 30, 2024 | 09:00 AM - 12:00 PM | Data Carpentry: Introduction to Python | [Register Here](https://cmu.libcal.com/

In [17]:
with open("./index.md", "w") as file:
    file.write(index_md)

In [18]:
with open("./README.md", "w") as file:
    file.write(index_md)